# SDXL-Turbo: Finding a reasonable CLAMP range for exploring the prompt embedding space
Using the diverse parti prompts v2 (1,6k entries) to explore the range of values for prompt embeddings. 

In [5]:
import torch
from evolutionary_prompt_embedding.image_creation import SDXLPromptEmbeddingImageCreator
from evolutionary_prompt_embedding.utils import clamp_range_from_parti, clamp_range_per_entry

In [2]:
creator = SDXLPromptEmbeddingImageCreator(batch_size=1, inference_steps=1)

Keyword arguments {'safety_checker': None, 'requires_safety_checker': False} are not expected by StableDiffusionXLPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loaded StableDiffusionXLPipeline {
  "_class_name": "StableDiffusionXLPipeline",
  "_diffusers_version": "0.25.0",
  "_name_or_path": "stabilityai/sdxl-turbo",
  "feature_extractor": [
    null,
    null
  ],
  "force_zeros_for_empty_prompt": true,
  "image_encoder": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "EulerAncestralDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}


## Simple min-max range for prompt_embeds and pooled_prompt_embeds

In [3]:
test1 = clamp_range_from_parti(creator, lambda_accessor=lambda x: x.prompt_embeds)
test2 = clamp_range_from_parti(creator, lambda_accessor=lambda x: x.pooled_prompt_embeds)
print("Range for prompt_embeds: ", test1)
print("Range for pooled_prompt_embeds: ", test2)

Token indices sequence length is longer than the specified maximum sequence length for this model (84 > 77). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (84 > 77). Running this sequence through the model will result in indexing errors


The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['as a beacon over rolling blue hills']
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['as a beacon over rolling blue hills']
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['as a beacon over rolling blue hills']
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['as a beacon over rolling blue hills']
Range for prompt_embeds:  (-809.0, 854.5)
Range for pooled_prompt_embeds:  (-8.0625, 7.8828125)


For the SDXL-Turbo model the CLAMP range for prompt_embeds is around (-810, 860) and for pooled_prompt_embeds it is around (-8, 8).
Keep in mind you can extend the values, but this restricts the search space to a reasonable range.

## More detailed CLAMP range for each entry in the tensor

In [10]:
min_tensor, max_tensor = clamp_range_per_entry(creator, lambda_accessor=lambda x: x.prompt_embeds)
print("prompt_embeds:")
print("Min tensor: ", min_tensor)
print("Max tensor: ", max_tensor)
torch.save(min_tensor.to('cpu'), 'sdxl_turbo_min_tensor.pt')
torch.save(max_tensor.to('cpu'), 'sdxl_turbo_max_tensor.pt')
min_tensor_pooled, max_tensor_pooled = clamp_range_per_entry(creator, lambda_accessor=lambda x: x.pooled_prompt_embeds)
print("pooled_prompt_embeds:")
print("Min tensor pooled: ", min_tensor_pooled)
print("Max tensor pooled: ", max_tensor_pooled)
torch.save(min_tensor_pooled.to('cpu'), 'sdxl_turbo_min_tensor_pooled.pt')
torch.save(max_tensor_pooled.to('cpu'), 'sdxl_turbo_max_tensor_pooled.pt')

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['as a beacon over rolling blue hills']
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['as a beacon over rolling blue hills']
prompt_embeds:
Min tensor:  tensor([[[-3.8926, -2.5117,  4.7148,  ...,  0.1899,  0.4180, -0.2959],
         [-1.6904, -1.4238, -1.5117,  ..., -1.5742, -1.9365, -1.4561],
         [-1.8291, -1.6348, -1.6777,  ..., -1.8506, -1.7236, -1.9873],
         ...,
         [-1.4150, -1.3574, -2.2207,  ..., -0.8662, -0.4707, -0.2817],
         [-1.4033, -1.3652, -2.2090,  ..., -0.9355, -0.5312, -0.3933],
         [-1.3906, -1.2686, -2.1445,  ..., -0.8530, -0.6299, -0.6133]]],
       device='mps:0', dtype=torch.float16)
Max tensor:  tensor([[[-3.8926, -2.5117,  4.7148,  ...,  0.1899,  0.4180, -0.2959],
         [ 1.5918,  1.7695,  1.2334,  ...,  1.8398,  2.2480,  1.3789],
         [ 1.9492,  1.7354,  1.7012,  ...,  2

In [11]:
diff_tensor = (max_tensor - min_tensor).to(dtype=torch.float32)
print("Value range: ", diff_tensor.sum())
diff_tensor_pooled = (max_tensor_pooled - min_tensor_pooled).to(dtype=torch.float32)
print("Value range pooled: ", diff_tensor_pooled.sum())

Value range:  tensor(542178.6250, device='mps:0')
Value range pooled:  tensor(7362.3359, device='mps:0')
